In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
import statistics as es 
import csv
import re 
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

## ENEM  DATASET

In [2]:
#Choose just useful features for us 
fenem= ['NU_INSCRICAO','CO_ESCOLA','Q001','Q002', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']
enem17=pd.read_csv("~/data/Enem/MICRODADOS_ENEM_2017.csv", sep=';',encoding="iso-8859-2", usecols = fenem)
enem18=pd.read_csv("~/data/Enem/MICRODADOS_ENEM_2018.csv", sep=';',encoding="iso-8859-2", usecols = fenem)

In [3]:
#category=(['NU_INSCRICAO','CO_ESCOLA','Q001','Q002'])
#enem17[category]= enem17[category].astype('category')

In [6]:
enem17 = enem17.copy()
enem18 = enem18.copy()


In [7]:
#How many Students left? How many schools are they in?
print ("There are", enem17.shape[0], "students",  "of", enem17.CO_ESCOLA.value_counts().count() ,"Schools in ENEM 2017")
#enem17.head()

print ("There are", enem18.shape[0], "students",  "of", enem18.CO_ESCOLA.value_counts().count() ,"Schools in ENEM 2018")
#enem18.head()

There are 6731341 students of 33834 Schools in ENEM 2017
There are 5513747 students of 32253 Schools in ENEM 2018


#### Set last quartile of average scores as the best students

In [8]:
#first, drop out those whithout schools 
enem17.dropna(inplace = True, axis=0, subset=['CO_ESCOLA'])
enem18.dropna(inplace = True, axis=0, subset=['CO_ESCOLA'])

In [9]:
#Second, drop out insufficient scores 
fields = (['NU_NOTA_CN', 'NU_NOTA_CN','NU_NOTA_CN','NU_NOTA_CN', 'NU_NOTA_REDACAO'])
# With NaN
enem17.dropna(inplace = True, axis=0, subset=fields)
enem18.dropna(inplace = True, axis=0, subset=fields)
# With 0
enem17 = enem17[~(enem17[fields] == 0).any(axis=1)]
enem18 = enem18[~(enem18[fields] == 0).any(axis=1)]

In [10]:
#Creating average of students in all fields
enem17['AVG_SCORE'] = (enem17.NU_NOTA_CN + enem17.NU_NOTA_CH + enem17.NU_NOTA_LC + enem17.NU_NOTA_MT + enem17.NU_NOTA_REDACAO)/5
enem18['AVG_SCORE'] = (enem18.NU_NOTA_CN + enem18.NU_NOTA_CH + enem18.NU_NOTA_LC + enem18.NU_NOTA_MT + enem18.NU_NOTA_REDACAO)/5

In [11]:
#Creating the average of all schools by each studentes score
enem17['AVG_SCORE_SCHOOL'] = enem17.groupby(['CO_ESCOLA'])['AVG_SCORE'].transform('mean') 
enem18['AVG_SCORE_SCHOOL'] = enem18.groupby(['CO_ESCOLA'])['AVG_SCORE'].transform('mean') 

In [12]:
#Sorting out the shcool's scores in quartiles. The upper quartile will be the Good Schools.
enem17['TARGET'] = pd.qcut (enem17.AVG_SCORE_SCHOOL, 4, labels = [1,2,3,4]).map(lambda x : 0 if x!=4 else 1) 
enem18['TARGET'] = pd.qcut (enem18.AVG_SCORE_SCHOOL, 4, labels = [1,2,3,4]).map(lambda x : 0 if x!=4 else 1) 


In [13]:
#It's right? 
print((enem17.TARGET==0).sum()/(enem17.TARGET.count())*100, '% lowers quartis in ENEM 2017')
print((enem18.TARGET==0).sum()/(enem18.TARGET.count())*100, '% lowers quartis in ENEM 2018')

75.00009542589339 % lowers quartis in ENEM 2017
75.00310581817226 % lowers quartis in ENEM 2018


## TEACHERS CENSUS DATASET 

In [19]:
#Choose just useful features for us now, improve memory use
fcenso = ['CO_PESSOA_FISICA','NU_ANO_CENSO','IN_ESPECIALIZACAO','IN_MESTRADO','IN_DOUTORADO','CO_ENTIDADE']

#Reading data of Teachers in Censo 2017
CO17=pd.read_csv("~/data/censo2017/DOCENTES_CO.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso)
NE17=pd.read_csv("~/data/censo2017/DOCENTES_NORDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso)
NT17=pd.read_csv("~/data/censo2017/DOCENTES_NORTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso)
SE17=pd.read_csv("~/data/censo2017/DOCENTES_SUDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso)
SU17=pd.read_csv("~/data/censo2017/DOCENTES_SUL.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso)

#There are differences in feature names
fcenso18 = ['ID_DOCENTE','NU_ANO_CENSO','IN_ESPECIALIZACAO','IN_MESTRADO','IN_DOUTORADO','CO_ENTIDADE']

CO18=pd.read_csv("~/data/censo2018/DOCENTES_CO.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso18)
NE18=pd.read_csv("~/data/censo2018/DOCENTES_NORDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso18)
NT18=pd.read_csv("~/data/censo2018/DOCENTES_NORTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso18)
SE18=pd.read_csv("~/data/censo2018/DOCENTES_SUDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso18)
SU18=pd.read_csv("~/data/censo2018/DOCENTES_SUL.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso18)


In [20]:
#Concatening the subsets and creating a unique dataset, than transform all features and categories, as it are.
censo17= pd.concat([CO17, NE17, NT17, SE17, SU17])
censo17 = censo17.copy()

censo18= pd.concat([CO18, NE18, NT18, SE18, SU18])
censo18 = censo18.copy()
#censo17=censo17.astype('category')

Get only teachers who work in secondary schools (based on ENEM)

In [23]:
#Some informations
print ("There are", censo17.drop_duplicates('CO_PESSOA_FISICA').shape[0], "Teachers",  "at", censo17.CO_ENTIDADE.value_counts().count() ,"Schools in Census 2017")
censo17.head()

print ("There are", censo18.drop_duplicates('ID_DOCENTE').shape[0], "Teachers",  "at", censo18.CO_ENTIDADE.value_counts().count() ,"Schools in Census 2018")
censo17.head()

There are 2603186 Teachers at 185925 Schools in Census 2017
There are 2581297 Teachers at 183746 Schools in Census 2018


,NU_ANO_CENSO,CO_PESSOA_FISICA,IN_ESPECIALIZACAO,IN_MESTRADO,IN_DOUTORADO,CO_ENTIDADE
0,2017,117579049642,1.0,0.0,0.0,53056000
1,2017,117579049642,1.0,0.0,0.0,53056000
2,2017,117579049642,1.0,0.0,0.0,53056000
3,2017,113802839063,0.0,0.0,0.0,53008456
4,2017,113802839063,0.0,0.0,0.0,53008456


In [25]:
#It's necessary drop duplicates whose are teachers in the same school but in different classes. 
#This way, the teacher appear just one time in each school. 
censo17.drop_duplicates(subset=['CO_PESSOA_FISICA', 'CO_ENTIDADE'], inplace=True)
censo18.drop_duplicates(subset=['ID_DOCENTE', 'CO_ENTIDADE'], inplace=True)

In [26]:
#getting the schools represented in ENEM
schoolsEnem17 = enem17.drop_duplicates('CO_ESCOLA')
schoolsEnem17 = schoolsEnem17[(['CO_ESCOLA','TARGET'])]
#schoolsEnem.astype('category', inplace=True)

schoolsEnem18 = enem18.drop_duplicates('CO_ESCOLA')
schoolsEnem18 = schoolsEnem18[(['CO_ESCOLA','TARGET'])]

In [27]:
#Merge the two datasets, whereas that only teachers who work in school represented in ENEM 17 and 18
censo17= pd.merge(schoolsEnem17, censo17, left_on='CO_ESCOLA', right_on='CO_ENTIDADE', how='inner')

censo18= pd.merge(schoolsEnem18, censo18, left_on='CO_ESCOLA', right_on='CO_ENTIDADE', how='inner')

In [29]:
print(censo17.isnull().sum())
print(censo18.isnull().sum())

CO_ESCOLA                 0
TARGET                    0
NU_ANO_CENSO              0
CO_PESSOA_FISICA          0
IN_ESPECIALIZACAO    117764
IN_MESTRADO          117764
IN_DOUTORADO         117764
CO_ENTIDADE               0
dtype: int64
CO_ESCOLA                0
TARGET                   0
NU_ANO_CENSO             0
ID_DOCENTE               0
IN_ESPECIALIZACAO    97672
IN_MESTRADO          97672
IN_DOUTORADO         97672
CO_ENTIDADE              0
dtype: int64


In [30]:
#Missing values in these cases are "0" 
censo17['IN_ESPECIALIZACAO'].fillna(0.0, inplace=True)
censo17['IN_MESTRADO'].fillna(0.0, inplace=True)
censo17['IN_DOUTORADO'].fillna(0.0, inplace=True)

censo18['IN_ESPECIALIZACAO'].fillna(0.0, inplace=True)
censo18['IN_MESTRADO'].fillna(0.0, inplace=True)
censo18['IN_DOUTORADO'].fillna(0.0, inplace=True)

In [32]:
#In database have different patterns to represent level of education. ex: Master has signed in two different ways, or marked in Master and Especialization or just in Master.
def education_level(df):
    if (df.IN_ESPECIALIZACAO==0.0) & (df.IN_MESTRADO==0.0) & (df.IN_DOUTORADO ==0.0):
        #print(0)
        df['TITULACAO']= 0
    elif (df.IN_ESPECIALIZACAO==1.0) & (df.IN_MESTRADO==0.0) & (df.IN_DOUTORADO ==0.0):
        #print(1)
        df['TITULACAO']= 1
    elif (df.IN_ESPECIALIZACAO==1.0) & (df.IN_MESTRADO==1.0) & (df.IN_DOUTORADO ==0.0):
        #print(2)
        df['TITULACAO']= 2
    elif (df.IN_ESPECIALIZACAO==0.0) & (df.IN_MESTRADO==1.0) & (df.IN_DOUTORADO ==0.0):
        #print(2)
        df['TITULACAO']= 2
    else:
        #print(3)
        df['TITULACAO']= 3  
    return df



In [33]:
censo17=censo17.apply(education_level, axis=1)

censo18=censo18.apply(education_level, axis=1)

##### We will transform the Categorical ENEM database features from student grain to school grain (Decision Grain).  Instead the pratical use of MODE, it will use the regression on the attribute distribution, having its histogram relative frequencies as input and the  decision level TARGET as output.  

In [ ]:
#Creating the dataframe with the relative frequencies and target. Ready to applying regeression
def relative_frequencies(df, colum):
    root = (['CO_ESCOLA', 'TARGET'])
    df = df [root + [colum]]
    df = pd.get_dummies(df, columns=[colum])
    filter_col = [col for col in df if col.startswith(colum)]
    print (filter_col)
    for x in filter_col:            
        df[x] = df.groupby(df['CO_ESCOLA'])[x].transform(sum)/ df.groupby(df['CO_ESCOLA'])[x].transform('count')
        
    return df


    

In [ ]:
def LR_student_to_school (df1, df2, columns):
    #colum = (['Q001', 'Q002'])
    result = pd.DataFrame()
    for x in columns:
        #print(df1[x])
        temp = relative_frequencies(df1,df1[x].name)
        temp.drop_duplicates('CO_ESCOLA', inplace=True)
        temp = temp.loc[:, ~temp.columns.isin(['CO_ESCOLA'])]
        temp_DATA = temp.loc[:, temp.columns != 'TARGET']
        temp_TARGET = temp.loc[:, temp.columns == 'TARGET']
        
        LR_temp = LogisticRegression()
        LR_temp.fit(temp_DATA, temp_TARGET)
        print("Aprendi")
        
        
        temp = relative_frequencies(df2,df2[x].name)
        temp.drop_duplicates('CO_ESCOLA', inplace=True) # v se funciona o inplace
        temp = temp.loc[:, ~temp.columns.isin(['CO_ESCOLA'])]
        temp_DATA = temp.loc[:, temp.columns != 'TARGET']
        
        result[x] =  LR_temp.predict_proba(temp_DATA)[:,1]
        print("Apliquei")
       
    result = pd.merge(result, df2.CO_ESCOLA, left_index=True, right_index=True)
        
    return result
        
        

        
        
        

In [ ]:
category_enem = (['Q001', 'Q002'])
trans_enem_att=LR_student_to_school(enem17, enem18, colum)

In [ ]:
data0 = pd.merge(trans_enem_att,transform_censo_att, left_on='CO_ESCOLA', right_on='CO_ENTIDADE', how='inner')

In [ ]:
Q001 = relative_frequencies(enem17, enem17.Q001.name)
Q001 = Q001.drop_duplicates_trLR_Q001 = LogisticRegression()
LR_Q001.fit(Q001_DATA, Q001_TARGET)



In [ ]:
Q002 = relative_frequencies(enem17, enem17.Q002.name)
Q002 = Q002.drop_duplicates('CO_ESCOLA')
Q002 = Q002.loc[:, ~Q002.columns.isin(['CO_ESCOLA'])]
Q002_DATA = Q002.loc[:, Q002.columns != 'TARGET']
Q002_TARGET = Q002.loc[:, Q002.columns == 'TARGET']

from sklearn.linear_model import LogisticRegression
LR_Q002 = LogisticRegression()
LR_Q002.fit(Q002_DATA,Q002_TARGET)


In [ ]:
TITULACAO = relative_frequencies(censo17, censo17.TITULACAO.name)
TITULACAO = TITULACAO.drop_duplicates('CO_ESCOLA')
TITULACAO = TITULACAO.loc[:, ~TITULACAO.columns.isin(['CO_ESCOLA'])]
TITULACAO_DATA =TITULACAO.loc[:, TITULACAO.columns != 'TARGET']
TITULACAO_TARGET = TITULACAO.loc[:, TITULACAO.columns == 'TARGET']

from sklearn.linear_model import LogisticRegression
LR_TITULACAO = LogisticRegression()
LR_TITULACAO.fit(TITULACAO_DATA, TITULACAO_TARGET)


In [ ]:
LR_Q002.predict_proba(Q002_DATA)[:,1]